In [1]:
# import des libraries

import requests
import pandas as pd
import json

In [2]:
# Essaies API

r = requests.get("https://nominatim.openstreetmap.org/search?q=Mont Saint Michel&format=json")
r.json()[0]

{'place_id': 258697296,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 376823,
 'boundingbox': ['48.6119741', '48.637031', '-1.5495487', '-1.5094805'],
 'lat': '48.6358559',
 'lon': '-1.5126096',
 'display_name': 'Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.8512740929650575,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'}

In [3]:
#  Création d'une liste de ville 

cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen",
          "Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg",
          "Colmar","Eguisheim","Besancon","Dijon","Annecy",
          "Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas",
          "Cassis","Marseille","Aix en Provence","Avignon","Uzes",
          "Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne",
          "Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

In [4]:
# Conversion des keys du fichier Json en liste

cols = list(r.json()[0])
cols

['place_id',
 'licence',
 'osm_type',
 'osm_id',
 'boundingbox',
 'lat',
 'lon',
 'display_name',
 'class',
 'type',
 'importance',
 'icon']

In [5]:
# Création d'un dataframe (data) vide à partir de cette liste

data = pd.DataFrame(columns=cols)
data

,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,importance,icon


In [6]:
# Utilisation de la liste des villes pour itérer sur l'API ,récupérer les coordonnées
# Gps de toutes les villes et les ajouter au dataframe (data)
# Reshape du dataframe pour garger uniquement les colonnes 'display_name','lat','lon'

for i in cities:
    r = requests.get("https://nominatim.openstreetmap.org/search?q={}&format=json".format(i))
    data = data.append(r.json()[0], ignore_index = True)
data = data[['display_name','lat','lon']]


In [7]:
# Reshape de la colonne display_name pour ne garder que le nom de la ville

data['display_name'] = data['display_name'].apply(lambda x : x.split(",")[0])
data.head()

,display_name,lat,lon
0,Le Mont-Saint-Michel,48.6358559,-1.5126096
1,Saint-Malo,48.6454528,-2.015418
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


### création du nouveau dataframe

In [8]:
# Dataframe

In [9]:
data.head()

,display_name,lat,lon
0,Le Mont-Saint-Michel,48.6358559,-1.5126096
1,Saint-Malo,48.6454528,-2.015418
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


In [10]:
# création de 2 listes pour itérer

In [11]:
lat = list(data.iloc[1:,1])
lon = list(data.iloc[1:,2])

In [12]:
# Création du dataframe
# Utilisation de l'API sur la première ville de la liste [cities] => 'Mont Saint Michel' pour créer la dataframe

s = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat=48.6358559&lon=-1.5126096&\
                exclude=hourly,current,minutely,alerts&appid=450c1541a1d61234fbc1e7cb7f9c542b")

# Navigation dans le fichier json pour récupérer la clé 'descriprion' pour chacun des 8 jours

cols = s.json()['daily']

weathers = []
for i in cols :
    weathers.append(i['weather'][0]['description'])

# Création d'un dictionnaire keys : day{} ; values : 'description'
# exemple {day1 : broken clouds}

weather_per_day = {}
for i,j in zip (weathers, range(1,9)):
    weather_per_day['day{}'.format(j)] = i
weather_per_day

# Conversion du dictionnaire en dataframe (data_days)

data_days = pd.DataFrame.from_dict(weather_per_day, orient="index").transpose()

data_days                 

,day1,day2,day3,day4,day5,day6,day7,day8
0,moderate rain,light rain,light rain,moderate rain,light rain,moderate rain,moderate rain,light rain


In [13]:
# Utilisation des liste [lat] et [lon] pour itérer sur l'api
# Conversion de chaque itération en dictionnaire sous le même format que ci-dessus
# Ajout du dictionnaire à notre dataframe (data_days)

for i,j in zip (lat, lon):
    s = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude=hourly,\
    current,minutely,alerts&appid=450c1541a1d61234fbc1e7cb7f9c542b".format(i,j))
    s = s.json()['daily']
    day_weathers = []
    for i in s :
        day_weathers.append(i['weather'][0]['description'])
        days = {}
        for i,j in zip (day_weathers, range(1,9)):
            days['day{}'.format(j)] = i
    data_days = data_days.append(days, ignore_index=True)
data_days.head()

,day1,day2,day3,day4,day5,day6,day7,day8
0,moderate rain,light rain,light rain,moderate rain,light rain,moderate rain,moderate rain,light rain
1,overcast clouds,light rain,light rain,moderate rain,light rain,moderate rain,moderate rain,light rain
2,light rain,light rain,light rain,moderate rain,light rain,moderate rain,light rain,overcast clouds
3,overcast clouds,light rain,light rain,moderate rain,light rain,moderate rain,light rain,light rain
4,overcast clouds,light rain,light rain,moderate rain,light rain,moderate rain,light rain,light rain


In [14]:
# création de deux listes
# [unique_values] = ensemble des valeurs uniques relatives à la météo
# [rate] = note relative à chaque météo

unique_values = []
for i in range(1,9):
    unique_val_day = data_days['day{}'.format(i)].unique()
    for i in unique_val_day:
        if i not in unique_values:
            unique_values.append(i)

rates = ['4','2','1','5','3','6','8','7','9','10','0']

In [15]:
# Création d'un dictionnaire {météo : note_correspondante}

weather_rate = {}
for i,j in zip (unique_values, rates):
    weather_rate[i] = j

In [16]:
# Modification des colonnes 'day{}', remplacement des valeur par la note relative à la météo de la cellule
# conversion des colonnes day{} en int pour pouvoir effectuer le calcul de la moyenne

for i in range(1,9):
    data_days['day{}'.format(i)] = data_days['day{}'.format(i)].map(weather_rate)
    data_days['day{}'.format(i)] = data_days['day{}'.format(i)].astype(int)

In [17]:
# Ajout d'une colonne moyenne qui sera la moyenne des notes des colonnes 'day{}' sur les 7 jours

data_days['average'] = data_days.mean(numeric_only=True, axis=1)


In [18]:
# Fusion des dataframe (data) et (data_days) à partir de leurs colonnes index

data = pd.merge(data, data_days, left_index=True, right_index=True)

In [19]:
# Tri des valeurs à partir de la colonne 'average' par ordre décroissant
data = data.sort_values('average', ascending=False)
data.head()

,display_name,lat,lon,day1,day2,day3,day4,day5,day6,day7,day8,average
15,Grenoble,45.1875602,5.7357819,6,5,7,1,9,9,4,9,6.250
14,Annecy,45.8992348,6.1288847,5,5,5,1,9,9,4,9,5.875
9,Château du Haut-Kœnigsbourg,48.249489800000006,7.34429620253195,3,2,2,1,9,9,4,9,4.875
18,Bormes-les-Mimosas,43.1506968,6.3419285,8,7,1,4,1,5,6,7,4.875
27,Collioure,42.52505,3.0831554,6,8,2,7,1,5,5,5,4.875


In [20]:
order = []
for i in range(35):
    order.append(i)
    
data.insert(0, "id", order, True) 

In [21]:
data.set_index("id", inplace=True)
data

,display_name,lat,lon,day1,day2,day3,day4,day5,day6,day7,day8,average
id,,,,,,,,,,,,
0,Grenoble,45.1875602,5.7357819,6,5,7,1,9,9,4,9,6.250
1,Annecy,45.8992348,6.1288847,5,5,5,1,9,9,4,9,5.875
2,Château du Haut-Kœnigsbourg,48.249489800000006,7.34429620253195,3,2,2,1,9,9,4,9,4.875
3,Bormes-les-Mimosas,43.1506968,6.3419285,8,7,1,4,1,5,6,7,4.875
4,Collioure,42.52505,3.0831554,6,8,2,7,1,5,5,5,4.875
5,Eguisheim,48.0447968,7.3079618,3,2,2,1,4,9,9,9,4.875
6,Gorges du Verdon,43.7497,6.32859,8,6,1,4,1,9,1,8,4.750
7,Colmar,48.0777517,7.3579641,3,2,2,1,1,9,9,9,4.500
8,Uzès,44.0121279,4.4196718,8,8,4,4,1,1,5,2,4.125


In [24]:
lat_lon = ['lat','lon']
for i in lat_lon:
    data[i] = data[i].astype(float)
    data[i] = data[i].apply(lambda x: round(x,4))
data.head()

,display_name,lat,lon,day1,day2,day3,day4,day5,day6,day7,day8,average
id,,,,,,,,,,,,
0,Grenoble,45.1876,5.7358,6,5,7,1,9,9,4,9,6.250
1,Annecy,45.8992,6.1289,5,5,5,1,9,9,4,9,5.875
2,Château du Haut-Kœnigsbourg,48.2495,7.3443,3,2,2,1,9,9,4,9,4.875
3,Bormes-les-Mimosas,43.1507,6.3419,8,7,1,4,1,5,6,7,4.875
4,Collioure,42.5251,3.0832,6,8,2,7,1,5,5,5,4.875


In [ ]:
data.to_csv("classement_météo.csv")

In [25]:
import boto3
session = boto3.Session(aws_access_key_id="***",\
                        aws_secret_access_key="***")

In [26]:
s3 = session.resource("s3")

In [34]:
bucket_name = s3.create_bucket(Bucket="kayak34654758342543647")

In [35]:
put_object = bucket_name.put_object(Key="city_weather.csv", Body=csv)

In [ ]:
csv = data.to_csv()